In [7]:
from pyspark.sql import SparkSession, functions as f

In [4]:
spark = SparkSession.builder.appName("batch").getOrCreate()

25/03/02 20:09:58 WARN Utils: Your hostname, DESKTOP-0NOFBU1 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/02 20:09:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 20:09:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/02 20:10:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read.parquet("./yellow_tripdata_2024-10.parquet")


[Stage 0:>                                                          (0 + 1) / 1]



In [ ]:
df.repartition(4).write.parquet("./yellow.parquet", mode="overwrite")

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [13]:
df.filter((f.day('tpep_pickup_datetime') == 15)).count()

128893

In [20]:
df = df.withColumn("delta_hours", f.expr("timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)"))
df.select("delta_hours").agg(f.max('delta_hours')).show()

+----------------+
|max(delta_hours)|
+----------------+
|             162|
+----------------+



In [29]:
zones = spark.read.option('header',True).csv('./taxi_zone_lookup.csv')
zones

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [27]:
taxi_zones = df.join(zones,how='left', on=f.col('PULocationID') == f.col('LocationID'))

In [31]:
taxi_zones.groupby('Zone').count().sort(f.asc('count')).show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|   Rossville/Woodrow|    4|
|       West Brighton|    4|
|Eltingville/Annad...|    4|
|       Port Richmond|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|Heartland Village...|    7|
|     Mariners Harbor|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|       Broad Channel|   10|
|New Dorp/Midland ...|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

